In [1]:
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import prettytensor as pt
from prettytensor.tutorial import data_utils
from function_face import *

/Users/allentang/Packages/anaconda/envs/tensorflow2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
BATCH_SIZE = 50
IMAGE_SIZE = 96
NUM_CHANNELS = 1
NUM_TARGETS = 30
VALIDATION_SIZE = 100

In [3]:
EPOCH_SIZE = 60000 // BATCH_SIZE
TEST_SIZE = 10000 // BATCH_SIZE

In [4]:
x_train, y_train, x_valid, y_valid, x_test = get_data(VALIDATION_SIZE)

Information on dataset
x_train (2040, 96, 96, 1)
y_train (2040, 30)
x_valid (100, 96, 96, 1)
y_valid (100, 30)
x_test (1783, 96, 96, 1)


In [5]:
x_train = x_train[0:2000]

In [6]:
y_train = y_train[0:2000]

In [7]:
def lenet5(images, labels):
    images = pt.wrap(images)
    with pt.defaults_scope(activation_fn=tf.nn.relu, l2loss=0.00001):
        return (images.conv2d(5, 20).max_pool(2, 2).conv2d(5, 50).max_pool(2, 2)
                .flatten().fully_connected(500)
                .fully_connected(NUM_TARGETS, activation_fn=tf.sigmoid)
                .cross_entropy(labels))

In [8]:
image_placeholder = tf.placeholder(tf.float32, [BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 1])
labels_placeholder = tf.placeholder(tf.float32, [BATCH_SIZE, NUM_TARGETS])

In [9]:
result = lenet5(image_placeholder, labels_placeholder)

In [10]:
accuracy = result
# result.evaluate_classifier(labels_placeholder, phase=pt.Phase.test)

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_op = pt.apply_optimizer(optimizer, losses=[result])
runner = pt.train.Runner()

In [ ]:
with tf.Session():
    for epoch in xrange(10):
#     train_images, train_labels = data_utils.permute_data(
#           (train_images, train_labels))
        runner.train_model(
          train_op,
          result,
          EPOCH_SIZE,
          feed_vars=(image_placeholder, labels_placeholder),
          feed_data=pt.train.feed_numpy(BATCH_SIZE, x_train, y_train),
          print_every=100)
        regression_loss = runner.evaluate_model(
          accuracy,
          VALIDATION_SIZE,
          feed_vars=(image_placeholder, labels_placeholder),
          feed_data=pt.train.feed_numpy(BATCH_SIZE, x_valid, y_valid))
        print('Accuracy after %d epoch %g%%' % (epoch + 1, regression_loss * 100))

[1] [50.720657]
[40] [50.667057]
Accuracy after 1 epoch 5111.34%
[41] [50.384483]
